In [1]:
# TODO - retrieve a representation by using the single encoders

import glob
import h5py
import hydra
import mmap
import numpy as np
import os
import pickle
import torch 
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torch.utils.data as data 

from omegaconf import DictConfig, OmegaConf
from collections import OrderedDict
from tqdm import tqdm 
from torch.nn.parallel import DistributedDataParallel as DDP
from holobot.robot.allegro.allegro_kdl import AllegroKDL

from tactile_learning.utils.constants import *
from tactile_learning.models.custom import *
from tactile_learning.datasets.tactile_vision import *
from tactile_learning.deployment.load_models import * 
from tactile_learning.deployment.nn_buffer import NearestNeighborBuffer
from tactile_learning.models.knn import KNearestNeighbors, ScaledKNearestNeighbors
from tactile_learning.utils.visualization import *
from tactile_learning.utils.tactile_image import *

In [7]:
TEST_ROOTS = ['/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_image_tactile_kinova_scaled_knn_3'] # TODO - change this to knn_4
REPR_DIR = '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/training'
REPR_ROOTS = sorted(glob.glob(f'{REPR_DIR}/demonstration_*'))

SINGLE_SENSOR_CNN_OUT_DIR = '/home/irmak/Workspace/tactile-learning/tactile_learning/out/2023.01.13/18-43_tactile_byol_bs_1028_tactile_play_data_single_sensor_encoder'
SINGLE_SENSOR_LINEAR_OUT_DIR = '/home/irmak/Workspace/tactile-learning/tactile_learning/out/2023.01.13/18-16_tactile_linear_byol_bs_1028_tactile_play_data_linear_encoder'
IMAGE_ENCODER_OUT_DIR = '/home/irmak/Workspace/tactile-learning/tactile_learning/out/2023.01.10/20-22_image_byol_bs_32_box_handle_lifting/box_location_changing/training'


In [8]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29506"

torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
torch.cuda.set_device(0)

RuntimeError: trying to initialize the default process group twice!

In [13]:
def init_encoder_info(device, out_dir, encoder_type='tactile'): # encoder_type: either image or tactile
    # print('out_dir: {}'.format(out_dir))
    cfg = OmegaConf.load(os.path.join(out_dir, '.hydra/config.yaml'))
    model_path = os.path.join(out_dir, 'models/byol_encoder_best.pt')
    encoder = load_model(cfg, device, model_path)
    encoder.eval() 
    if encoder_type == 'tactile':
        transform = T.Compose([
            T.Resize((4,4)),
            T.Normalize(TACTILE_IMAGE_MEANS, TACTILE_IMAGE_STDS),
        ])
    elif encoder_type == 'image':
        transform = T.Compose([
            T.Resize((480,640)),
            T.Lambda(crop_transform),
            T.ToTensor(),
            T.Normalize(VISION_IMAGE_MEANS, VISION_IMAGE_STDS),
        ]) 

    return cfg, encoder, transform

In [14]:
device = torch.device('cuda:0')
tactile_cfg, tactile_single_cnn, tactile_transform = init_encoder_info(device, SINGLE_SENSOR_CNN_OUT_DIR, 'tactile')
image_cfg, image_encoder, image_transform = init_encoder_info(device, IMAGE_ENCODER_OUT_DIR, 'image')

np_means = np.asarray(VISION_IMAGE_MEANS)
np_stds = np.asarray(VISION_IMAGE_STDS)

inv_image_transform = T.Compose([
    T.Normalize(mean = [0,0,0], std = 1 / np_stds ), 
    T.Normalize(mean = -np_means, std = [1,1,1])
])

cfg.learner_type: tactile_byol
cfg.learner_type: image_byol


In [25]:
def get_tactile_representation(tactile_values, tactile_encoder): # tactile_values.shape: (15,16,3)
    def _get_single_tactile_image(tactile_value):
        tactile_image = torch.FloatTensor(tactile_value) # tactile_value.shape: (16,3)
        tactile_image = tactile_image.view(4,4,3)
        tactile_image = torch.permute(tactile_image, (2,0,1))
        return tactile_transform(tactile_image)

    for sensor_id in range(len(tactile_values)):
        curr_tactile_value = tactile_values[sensor_id]
        curr_tactile_image = _get_single_tactile_image(curr_tactile_value).unsqueeze(0) # To make it as if it's a batch
        if sensor_id == 0:
            curr_repr = tactile_encoder(curr_tactile_image).squeeze() # shape: (64)
        else:
            curr_repr =  torch.cat([curr_repr, tactile_encoder(curr_tactile_image).squeeze()], dim=0)
        print('curr_repr.shape: {}'.format(curr_repr.shape))

    return curr_repr

In [22]:
repr_data = load_data(REPR_ROOTS)
test_data = load_data(TEST_ROOTS)

In [26]:
DEMO_ID = 0
TACTILE_ID = 2000 #2126
# SENSOR_ID = 7
tactile_values = test_data['tactile']['values'][DEMO_ID][TACTILE_ID]
tactile_repr = get_tactile_representation(tactile_values, tactile_single_cnn)

curr_repr.shape: torch.Size([64])
curr_repr.shape: torch.Size([128])
curr_repr.shape: torch.Size([192])
curr_repr.shape: torch.Size([256])
curr_repr.shape: torch.Size([320])
curr_repr.shape: torch.Size([384])
curr_repr.shape: torch.Size([448])
curr_repr.shape: torch.Size([512])
curr_repr.shape: torch.Size([576])
curr_repr.shape: torch.Size([640])
curr_repr.shape: torch.Size([704])
curr_repr.shape: torch.Size([768])
curr_repr.shape: torch.Size([832])
curr_repr.shape: torch.Size([896])
curr_repr.shape: torch.Size([960])


In [27]:
tactile_repr.shape

torch.Size([960])